In [204]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from sklearn.metrics import classification_report
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [205]:
df_elasticidad = pd.read_pickle("../output/base_etapa_3.pkl")

In [218]:
# Creamos una columna para simular el costo
df_elasticidad['costo'] = 17
df_elasticidad['costo_seg0'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg0']))
df_elasticidad['costo_seg1'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg1']))
df_elasticidad['costo_seg2'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg2']))
df_elasticidad['costo_seg3'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg3']))
df_elasticidad['costo_seg4'] = np.exp(np.log(df_elasticidad['costo'])*np.log(df_elasticidad['cantidad_seg4']))

In [219]:
df_elasticidad['ingresos_seg0'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg0']))
df_elasticidad['ingresos_seg1'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg1']))
df_elasticidad['ingresos_seg2'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg2']))
df_elasticidad['ingresos_seg3'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg3']))
df_elasticidad['ingresos_seg4'] = np.exp(np.log(df_elasticidad['irr'])*np.log(df_elasticidad['cantidad_seg4']))

In [220]:
# Calculamos la rentabilidad
df_elasticidad['rentabilidad_seg0'] = df_elasticidad['ingresos_seg0'] - df_elasticidad["costo_seg0"]
df_elasticidad['rentabilidad_seg1'] = df_elasticidad['ingresos_seg1'] - df_elasticidad["costo_seg1"]
df_elasticidad['rentabilidad_seg2'] = df_elasticidad['ingresos_seg2'] - df_elasticidad["costo_seg2"]
df_elasticidad['rentabilidad_seg3'] = df_elasticidad['ingresos_seg3'] - df_elasticidad["costo_seg3"]
df_elasticidad['rentabilidad_seg4'] = df_elasticidad['ingresos_seg4'] - df_elasticidad["costo_seg4"]

# df_elasticidad['rentabilidad_seg0'] = df_elasticidad['ingresos_seg0'] - df_elasticidad["costo"]
# df_elasticidad['rentabilidad_seg1'] = df_elasticidad['ingresos_seg1'] - df_elasticidad["costo"]
# df_elasticidad['rentabilidad_seg2'] = df_elasticidad['ingresos_seg2'] - df_elasticidad["costo"]
# df_elasticidad['rentabilidad_seg3'] = df_elasticidad['ingresos_seg3'] - df_elasticidad["costo"]

In [221]:
df_elasticidad

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,cantidad_seg3,cantidad_seg4,costo,costo_seg0,costo_seg1,costo_seg2,...,ingresos_seg0,ingresos_seg1,ingresos_seg2,ingresos_seg3,ingresos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
0,16.00,2.434245,2.448691,2.474350,2.467429,2.444356,17,12.435197,12.645411,13.024441,...,11.782283,11.977163,12.328370,12.233003,11.918470,-0.652913,-0.668248,-0.696071,-0.688492,-0.663621
1,16.58,2.252067,2.266231,2.285492,2.280445,2.262313,17,9.975583,10.154355,10.400779,...,9.775029,9.948646,10.187921,10.124868,9.900427,-0.200554,-0.205709,-0.212858,-0.210969,-0.204274
2,17.16,2.089104,2.102958,2.116820,2.113349,2.099408,17,8.063346,8.215768,8.370128,...,8.119188,8.273178,8.429135,8.389909,8.233532,0.055842,0.057410,0.059007,0.058605,0.057005
3,17.74,1.942779,1.956306,1.965600,1.963459,1.953079,17,6.563944,6.694261,6.784758,...,6.752338,6.888431,6.982964,6.961107,6.855803,0.188394,0.194171,0.198206,0.197271,0.192782
4,18.32,1.810928,1.824118,1.829540,1.828522,1.821178,17,5.378847,5.490587,5.536953,...,5.623090,5.743020,5.792803,5.783429,5.716140,0.244243,0.252433,0.255850,0.255206,0.250592
5,18.89,1.693685,1.706539,1.708726,1.708642,1.703849,17,4.449709,4.546053,4.562581,...,4.703867,4.809545,4.827683,4.826985,4.787302,0.254158,0.263492,0.265102,0.265040,0.261521
6,19.47,1.585407,1.597918,1.597304,1.598026,1.595455,17,3.690130,3.773227,3.769121,...,3.928195,4.020938,4.016353,4.021746,4.002572,0.238065,0.247711,0.247232,0.247796,0.245794
7,20.05,1.486933,1.499102,1.496105,1.497510,1.496843,17,3.077075,3.148958,3.131160,...,3.285254,3.366525,3.346391,3.355820,3.351341,0.208178,0.217566,0.215232,0.216324,0.215805
8,20.63,1.397128,1.408959,1.403937,1.405919,1.406884,17,2.579199,2.641563,2.614973,...,2.751648,2.822785,2.792439,2.804391,2.810217,0.172449,0.181222,0.177467,0.178943,0.179664
9,21.21,1.315016,1.326516,1.319771,1.322242,1.324605,17,2.172486,2.226747,2.194818,...,2.308190,2.370403,2.333780,2.347152,2.359987,0.135704,0.143656,0.138962,0.140672,0.142317


In [222]:
# Graficamos las curvas de Precio-Ingresos
fig = px.line(df_elasticidad, x='irr', y=['ingresos_seg0', 'ingresos_seg1',
                                             'ingresos_seg2', 'ingresos_seg3', 'ingresos_seg4'],
              labels={'irr':'tasa', 'value':'Ingresos'},
              title='Curva de Ingresos',
              width=800, height=600)

fig.show()

In [223]:
# Graficamos las curvas de Precio-Rentabilidad
fig = px.line(df_elasticidad, x='irr', y=['rentabilidad_seg0',
                                             'rentabilidad_seg1',
                                             'rentabilidad_seg2',
                                             'rentabilidad_seg3',
                                             'rentabilidad_seg4',
                                             ],
              labels={'irr':'tasa', 'value':'Rentabilidad'},
              title='Curvas de Rentabilidad',
              width=800, height=600)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="red", opacity=0.5)

fig.show()

In [224]:
df_elasticidad.head()

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,cantidad_seg3,cantidad_seg4,costo,costo_seg0,costo_seg1,costo_seg2,...,ingresos_seg0,ingresos_seg1,ingresos_seg2,ingresos_seg3,ingresos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
0,16.00,2.434245,2.448691,2.474350,2.467429,2.444356,17,12.435197,12.645411,13.024441,...,11.782283,11.977163,12.328370,12.233003,11.918470,-0.652913,-0.668248,-0.696071,-0.688492,-0.663621
1,16.58,2.252067,2.266231,2.285492,2.280445,2.262313,17,9.975583,10.154355,10.400779,...,9.775029,9.948646,10.187921,10.124868,9.900427,-0.200554,-0.205709,-0.212858,-0.210969,-0.204274
2,17.16,2.089104,2.102958,2.116820,2.113349,2.099408,17,8.063346,8.215768,8.370128,...,8.119188,8.273178,8.429135,8.389909,8.233532,0.055842,0.057410,0.059007,0.058605,0.057005
3,17.74,1.942779,1.956306,1.965600,1.963459,1.953079,17,6.563944,6.694261,6.784758,...,6.752338,6.888431,6.982964,6.961107,6.855803,0.188394,0.194171,0.198206,0.197271,0.192782
4,18.32,1.810928,1.824118,1.829540,1.828522,1.821178,17,5.378847,5.490587,5.536953,...,5.623090,5.743020,5.792803,5.783429,5.716140,0.244243,0.252433,0.255850,0.255206,0.250592


In [225]:
# Hallamos los precios que maximicen los ingresos en cada segmento
max_ing_seg0 = df_elasticidad.loc[(df_elasticidad['ingresos_seg0'] == df_elasticidad['ingresos_seg0'].max())]
max_ing_seg1 = df_elasticidad.loc[(df_elasticidad['ingresos_seg1'] == df_elasticidad['ingresos_seg1'].max())]
max_ing_seg2 = df_elasticidad.loc[(df_elasticidad['ingresos_seg2'] == df_elasticidad['ingresos_seg2'].max())]
max_ing_seg3 = df_elasticidad.loc[(df_elasticidad['ingresos_seg3'] == df_elasticidad['ingresos_seg3'].max())]
max_ing_seg4 = df_elasticidad.loc[(df_elasticidad['ingresos_seg4'] == df_elasticidad['ingresos_seg4'].max())]

In [226]:
max_ing_seg0

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,cantidad_seg3,cantidad_seg4,costo,costo_seg0,costo_seg1,costo_seg2,...,ingresos_seg0,ingresos_seg1,ingresos_seg2,ingresos_seg3,ingresos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
0,16.0,2.434245,2.448691,2.47435,2.467429,2.444356,17,12.435197,12.645411,13.024441,...,11.782283,11.977163,12.32837,12.233003,11.91847,-0.652913,-0.668248,-0.696071,-0.688492,-0.663621


In [227]:
# Hallamos los precios que maximicen la rentabilidad en cada segmento
max_ren_seg0 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg0'] == df_elasticidad['rentabilidad_seg0'].max())]
max_ren_seg1 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg1'] == df_elasticidad['rentabilidad_seg1'].max())]
max_ren_seg2 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg2'] == df_elasticidad['rentabilidad_seg2'].max())]
max_ren_seg3 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg3'] == df_elasticidad['rentabilidad_seg3'].max())]
max_ren_seg4 = df_elasticidad.loc[(df_elasticidad['rentabilidad_seg4'] == df_elasticidad['rentabilidad_seg4'].max())]

In [228]:
max_ren_seg0

,irr,cantidad_seg0,cantidad_seg1,cantidad_seg2,cantidad_seg3,cantidad_seg4,costo,costo_seg0,costo_seg1,costo_seg2,...,ingresos_seg0,ingresos_seg1,ingresos_seg2,ingresos_seg3,ingresos_seg4,rentabilidad_seg0,rentabilidad_seg1,rentabilidad_seg2,rentabilidad_seg3,rentabilidad_seg4
5,18.89,1.693685,1.706539,1.708726,1.708642,1.703849,17,4.449709,4.546053,4.562581,...,4.703867,4.809545,4.827683,4.826985,4.787302,0.254158,0.263492,0.265102,0.26504,0.261521


In [229]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg0'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg0['irr']), y=float(max_ing_seg0['ingresos_seg0']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg0['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg0'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg0['irr']), y=float(max_ren_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg0['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 0")


In [230]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg1'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg1['irr']), y=float(max_ing_seg1['ingresos_seg1']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg1['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg1'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg1['irr']), y=float(max_ren_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg1['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 1")

In [231]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg2'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg2['irr']), y=float(max_ing_seg2['ingresos_seg2']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg2['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg2'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg2['irr']), y=float(max_ren_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg2['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 2")

In [232]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg3'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg3['irr']), y=float(max_ing_seg3['ingresos_seg3']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg3'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg3['irr']), y=float(max_ren_seg3['rentabilidad_seg3']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 3")

In [233]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['ingresos_seg4'], name='Ingresos ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ing_seg3['irr']), y=float(max_ing_seg3['ingresos_seg4']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega el precio máximo de rentabilidad
fig.add_trace(go.Scatter(x=df_elasticidad['irr'], y=df_elasticidad['rentabilidad_seg4'], name='Rentabilidad ($)'))
# Se agrega la etiqueta
fig.add_annotation(x=float(max_ren_seg3['irr']), y=float(max_ren_seg3['rentabilidad_seg4']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg3['irr']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Agregamos la línea de restricción: Costo
fig.add_vline(x=float(df_elasticidad['costo'].mean()), line_width=2, line_dash="dash",
                             line_color="orange", opacity=0.5)

# Configuración del gráfico
fig.update_layout(
    showlegend=True,
    width=800,
    height=600,
    xaxis_title="Precio ($)",
    yaxis_title="Importe ($)",
    title="Precios óptimos en Segmento 4")

In [200]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg0'], y=df_elasticidad['rentabilidad_seg0'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg0['ingresos_seg0']), y=float(max_ing_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg0['ingresos_seg0']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg0['ingresos_seg0']), y=float(max_ren_seg0['rentabilidad_seg0']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg0['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg0['ingresos_seg0']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis=dict(
        title="Ingreso ($)"
       ## ,range=[1, 2]
        ),
   yaxis=dict(
        title="Rentabilidad ($)"
       ##,range=[-0.5, 1]
    ),
    title="Frontera Eficiente Segmento 0")

In [201]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg1'], y=df_elasticidad['rentabilidad_seg1'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg1['ingresos_seg1']), y=float(max_ing_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg1['ingresos_seg1']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg1['ingresos_seg1']), y=float(max_ren_seg1['rentabilidad_seg1']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg1['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg1['ingresos_seg1']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
     xaxis=dict(
        title="Ingreso ($)"
      ##  ,range=[1.5, 2.5]
        ),
      yaxis=dict(
        title="Rentabilidad ($)"
       ##,range=[-0.5, 1]
      ),
    title="Frontera Eficiente Segmento 1")

In [202]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg2'], y=df_elasticidad['rentabilidad_seg2'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg2['ingresos_seg2']), y=float(max_ing_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg2['ingresos_seg2']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg2['ingresos_seg2']), y=float(max_ren_seg2['rentabilidad_seg2']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg2['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg2['ingresos_seg2']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis_title="Ingreso ($)",
    yaxis_title="Rentabilidad ($)",
    title="Frontera Eficiente Segmento 2")

In [203]:
fig = go.Figure()

# Se agrega el precio máximo de ingreso
fig.add_trace(go.Scatter(x=df_elasticidad['ingresos_seg3'], y=df_elasticidad['rentabilidad_seg3'], name='Frontera Eficiente'))

# Se agrega la etiqueta de ingreso
fig.add_annotation(x=float(max_ing_seg3['ingresos_seg3']), y=float(max_ing_seg3['rentabilidad_seg3']),
                                  text="MaxPrecioIng:"+str(float(max_ing_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ing_seg3['ingresos_seg3']), line_width=2, line_dash="dash",
                             line_color="blue", opacity=0.25)

# Se agrega la etiqueta de rentabilidad
fig.add_annotation(x=float(max_ren_seg3['ingresos_seg3']), y=float(max_ren_seg3['rentabilidad_seg3']),
                                  text="MaxPrecioRent:"+str(float(max_ren_seg3['irr'].round(2))),
                                  showarrow=True,
                                  arrowhead=1)
# Se agrega la recta del precio máximo
fig.add_vline(x=float(max_ren_seg3['ingresos_seg3']), line_width=2, line_dash="dash",
                             line_color="green", opacity=0.25)

# Configuración del gráfico
fig.update_layout(
    showlegend=False,
    width=800,
    height=600,
    xaxis_title="Ingreso ($)",
    yaxis_title="Rentabilidad ($)",
    title="Frontera Eficiente Segmento 3")